## Libraries

In [3]:
import numpy as np
import mediapipe as mp
import cv2
from collections import defaultdict

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## Angle Calculation

In [1]:
#defining a function that calculates angles giving coordinates as inputs

def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b) 
    c = np.array(c) 
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

## Triple Extension Checker

In [4]:
## using the previous functions together with pre-trained MediaPipe models to estimate poses so to calculate an angle in
## hip - knee - ankle to verify if there's a triple extension or not (> 160 degrees)

def triple_extension_checker(video):

    cap = cv2.VideoCapture(video)
        
    # checker variables
    counter = "not extended"
    stage = "" 
    
    ## set up mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
                
            # recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
              
            # rake detection
            results = pose.process(image)
            
            # recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image.flags.writeable = True
            
            # extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                    
                # get coordinates
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                    
                # calculate angle
                angle = calculate_angle(ankle, knee, hip)
                    
                # setting up tiple extension angle criteria
                if angle > 160:
                    stage = "" 
                    counter = "Perfect"
                               
            except:
                pass
                
            # estension status box
            cv2.rectangle(image, (0,0), (250,73), (245,117,16), -1)
                
            # status data
            cv2.putText(image, 'EXTENSION STATUS', (15,25), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                
            # stage data
            cv2.putText(image, '', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)  

            # Render detections
            #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
             #                       mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
              #                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
               #                      )  

            ### uncomment if you want to see the jointa estimations
        
            vid = cv2.imshow('Extension Checker', image)
        
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

    return vid
    
triple_extension_checker('video.mp4')

I0000 00:00:1702135474.329847       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
